In [1]:
import pandas as pd
import os 
import glob
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import datetime 

# path of the folder
folder_path = '/Users/jud-05/Desktop/High Court'
new_path = '/Users/jud-05/Desktop/High Court/combined'
path = '/Users/jud-05/Desktop/High Court/combined/dcrt'


# Ensure the new directory exists
os.makedirs(new_path, exist_ok=True)

# reading all the excel files
file_names = glob.glob(folder_path + "/*.xls")

file_names

#create a file name without the directory and the extension and name each as the first word of the file
file_names2 = [os.path.basename(f) for f in file_names]
file_names2 = [f.split('.')[0] for f in file_names2]
file_names2 = [f.split()[0] for f in file_names2]
file_names2


#save files names as csv files and rename each file with the name of the original file
for f in file_names:
    df = pd.read_excel(f, skiprows=4)  # Skip the first 4 rows
    # Extract the base name of the file and save it as a .csv in the new directory
    base_name = os.path.basename(f)
    df.to_csv(f'{new_path}/{base_name}.csv', index=False)
    

In [2]:
# Create a file name without the directory and the extension, appending a unique identifier
file_names2 = [os.path.basename(f).split('.')[0].split()[0] + '_' + str(i) for i, f in enumerate(file_names)]

# Save files as csv files
for f, base_name in zip(file_names, file_names2):
    df = pd.read_excel(f, skiprows=4)  # Skip the first 4 rows
    # Save it as a .csv in the new directory
    df.to_csv(f'{path}/{base_name}.csv', index=False)

In [3]:
# Get all the CSV files in the directory
all_files = glob.glob(path + "/*.csv")

all_files

# Combine all the CSV files into a single DataFrame and insert a column with the name of the file
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['file_name'] = os.path.basename(filename)
    li.append(df)
       
combined_csv = pd.concat(li, axis=0, ignore_index=True)


#save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_10714/2607724255.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_csv = pd.concat(li, axis=0, ignore_index=True)


In [4]:
combined_csv['file_name'] = combined_csv['file_name'].str.replace('.csv', '')

#remove in file_name the letters after the underscore
combined_csv['file_name'] = combined_csv['file_name'].str.split('_').str[0]
 #insert 

#move the file_name column to the first column
cols = combined_csv.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_csv = combined_csv[cols]

#drop Unnamed: 0 column
combined_csv = combined_csv.drop('Unnamed: 0', axis=1)

 #save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

In [5]:
#rename columns
# Simplified version of data.rename() function
combined_csv.rename(columns={
    'file_name': 'court_name',
    'Day': 'date_dd',
    'Month': 'date_mon',
    'Year': 'date_yyyy',
    'Code': 'caseid_type',
    'No.': 'caseid_no',
    'Day.1': 'filed_dd',
    'Month.1': 'filed_mon',
    'Year.1': 'filed_yyyy',
    'Name of Court': 'origcourt',
    'Code.1': 'appeal_type',
    'No..1': 'appeal_no',
    'Year.2': 'appeal_yyyy',
    '\n(Select)': 'case_type',
    'Name of Judge 1 or Magistrate/DR or Kadhi': 'bench_1',
    'Judge 2': 'bench_2',
    'Judge 3': 'bench_3',
    'Judge 4': 'bench_4',
    'Judge 5': 'bench_5',
    'Judge 6': 'bench_6',
    'Judge 7': 'bench_7',
    '(Select option)': 'comingfor',
    '(Select option).1': 'outcome',
    '(Select option).2': 'reason_adj',
    'Day.2': 'next_dd',
    'Month.2': 'next_mon',
    'Year.3': 'next_yyyy',
    'M': 'firstparty_m',
    'F': 'firstparty_f',
    'Org.': 'firstparty_org',
    'M.1': 'secondparty_m',
    'F.1': 'secondparty_f',
    'Org..1': 'secondparty_o',
    '(Y/N)': 'legalrep',
    'P \n(Enter No.)': 'p_witness',
    'D\n(Enter No.)': 'd_witness',
    '(Enter No.)': 'custody',
    '(Free text)': 'other_details'
}, inplace=True)

#save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

In [6]:
def convert_to_datetime(combined_csv, day_col, month_col, year_col, new_col, index):
    date_str = combined_csv[day_col].astype(str) + "-" + combined_csv[month_col] + "-" + combined_csv[year_col].astype(str)
    combined_csv[new_col] = pd.to_datetime(date_str, format="%d-%b-%Y", errors="coerce")
    #data.insert(index, new_col, data.pop(new_col))
    return combined_csv

combined_csv = convert_to_datetime(combined_csv, "date_dd", "date_mon", "date_yyyy", "activity_date", 2)
combined_csv = convert_to_datetime(combined_csv, "filed_dd", "filed_mon", "filed_yyyy", "filed_date", 3)
for i in combined_csv['next_dd'].unique():
    if i == " ":
        combined_csv.loc[combined_csv[['next_dd', 'next_mon', 'next_yyyy']] == i, 'next_dd', 'next_mon', 'next_yyyy'] = ""
    else:
        combined_csv = convert_to_datetime(combined_csv, "next_dd", "next_mon", "next_yyyy", "next_date", 4)
        
combined_csv['activity_date'].value_counts()

activity_date
2024-03-18    5875
2024-03-19    5737
2023-10-12    5617
2023-11-30    5547
2023-12-14    5485
              ... 
2023-09-10       1
2023-12-10       1
2023-07-02       1
2023-09-03       1
2023-07-30       1
Name: count, Length: 271, dtype: int64

In [7]:
#create unique case id
combined_csv["case_identifier"] = combined_csv ['court_name'] + "-" + combined_csv["caseid_type"]+ "-" + combined_csv["caseid_no"].astype(str) + "/" + combined_csv["filed_date"].astype(str)
combined_csv.insert(1, "case_identifier", combined_csv.pop("case_identifier"))

combined_csv["case_identifier"].value_counts()

case_identifier
Milimani-HCCOMM-E484/2022-12-06        59
Milimani-HCCOMMIP-E063/2021-09-28      59
Milimani-HCCOMM-E440/2023-09-13        52
Milimani-HCCOMM-506/2012-01-01         48
Milimani-HCCOMMARB-E042/2021-10-29     46
                                       ..
Eldoret-HCCC-52/2018-06-15              1
Eldoret-HCCC-4/2018-03-13               1
Eldoret-HCCC-68/2018-11-19              1
Eldoret-HCCC-65/2018-11-15              1
Kitale-HCCRMISCAPPL-E128/2023-03-06     1
Name: count, Length: 106803, dtype: int64

In [8]:
#check duplicates
combined_csv.duplicated().sum()

#drop duplicates
combined_csv = combined_csv.drop_duplicates()



In [9]:
combined_csv['court_name'].value_counts()

court_name
Milimani      117168
Mombasa        19124
Eldoret        12467
Kiambu         12053
Nakuru         11923
Meru           10468
Machakos        9089
Kakamega        8458
Kisumu          7272
Bungoma         7037
Muranga         6697
Nyeri           6350
Kitale          6171
Kisii           6090
Makueni         5389
Malindi         4760
Naivasha        4676
Kerugoya        4451
Kajiado         4192
Embu            3876
Kericho         3845
Migori          3320
Kitui           3158
Thika           3137
Homabay         3095
Busia           2844
Voi             2810
Siaya           2793
Kibera          2717
Bomet           2694
Kapsabet        2551
Narok           2394
Nanyuki         2362
Lodwar          2342
Garissa         2213
Nyamira         2166
Nyahururu       1896
Chuka           1847
Vihiga          1836
Kabarnet        1832
Nyandarua       1618
Kapenguria      1442
Iten            1106
Garsen          1074
Kilgoris         867
Marsabit         828
Kwale            785
El

In [10]:
combined_csv['case_type'].value_counts()

case_type
Civil Appeal                   50839
Family P&A Intestate           38158
Civil Case Miscellaneous       31649
Murder Case                    28248
Criminal Appeal                24327
                               ...  
JR  Petition Miscellaneous         2
HCC(OS) Family                     2
Family                             2
Citation-Family                    1
Gender Justice Civil Appeal        1
Name: count, Length: 61, dtype: int64

In [11]:
def create_case_nature(combined_csv, old_col, new_col, index):
    case_nature = combined_csv[old_col].astype(str).apply(lambda x: 
        "CRIMINAL" if any(y in x for y in ["CR"]) 
        else "CIVIL")
    combined_csv[new_col] = case_nature
   #combined_csv.insert(index, new_col, combined_csv.pop(new_col))
    return combined_csv
combined_csv = create_case_nature(combined_csv, "caseid_type", "case_nature", 5)




In [12]:
combined_csv['outcome'].value_counts()

outcome
Mention date set (in court)         100495
Directions given                     56486
Case Registered/Filed                36751
Hearing date set (in court)          23451
Adjournment                          15732
                                     ...  
Appeal Dismissed                         1
Extension of  Mediation Period           1
Matter settled through Mediation         1
Retrial                                  1
Inheritance Determined                   1
Name: count, Length: 101, dtype: int64

In [13]:
#generate function to generate filed cases width activity date before filed date
def filed_cases(combined_csv):
    filed = []
    for i, row in combined_csv.iterrows():
        if "Case Registered/Filed" in str(row['outcome']) and row['activity_date'] >= row['filed_date']:
            filed.append(1)
        else:
            filed.append(0)
    combined_csv['filed'] = filed
    return combined_csv

combined_csv = filed_cases(combined_csv)

combined_csv['filed'].value_counts()

filed
0    291717
1     36637
Name: count, dtype: int64

In [14]:
combined_csv['outcome'].value_counts()

outcome
Mention date set (in court)         100495
Directions given                     56486
Case Registered/Filed                36751
Hearing date set (in court)          23451
Adjournment                          15732
                                     ...  
Appeal Dismissed                         1
Extension of  Mediation Period           1
Matter settled through Mediation         1
Retrial                                  1
Inheritance Determined                   1
Name: count, Length: 101, dtype: int64

In [15]:
#change outcome column to upper case
combined_csv['outcome'] = combined_csv['outcome'].str.upper()
def resolved_cases(combined_csv):
    resolved = []
    for i in combined_csv['outcome'].astype(str):
        if ("ABATED" in i or "CASE CLOSED" in i or "STRUCK OUT" in i or  "TERMINATED" in i or "DISMISS" in i or "GRANT CONFIRMED" in i or "GRANT REVOKED" in i or "JUDGMENT DELIVERED" in i or "ACQUITTAL" in i or "CONVICTED" in i or "LIMITED GRANT ISSUED" in i or "WITHDRAW" in i or "SETTLEMEN" in i or "REVISION DECLINED" in i or "RULING DELIVERED- ACCUSED DISC" in i or "RULING DELIVERED- CASE CLOSED" in i) and ("ORDER ISSUED - CASE CLOSED" not in i):
            resolved.append(1)
    
        else:
            resolved.append(0)
    combined_csv['resolved'] = resolved
    return combined_csv

combined_csv = resolved_cases(combined_csv)

total_resolved = combined_csv['resolved'].sum()
combined_csv[['outcome', 'resolved']] [combined_csv['resolved'] == 1].value_counts()
total_resolved






35239

In [16]:
#resolved by court name
resolved_court = combined_csv.groupby(['court_name'])['resolved'].sum().reset_index()
resolved_court['resolved'] = resolved_court['resolved'].astype(int)

resolved_court = resolved_court.sort_values(by=['court_name'], ascending=True)

resolved_data = combined_csv[combined_csv['resolved'] == 1]

In [17]:
combined_csv

,court_name,case_identifier,date_dd,date_mon,date_yyyy,caseid_type,caseid_no,filed_dd,filed_mon,filed_yyyy,...,p_witness,d_witness,custody,other_details,activity_date,filed_date,next_date,case_nature,filed,resolved
0,Voi,Voi-HCFP&A-10/2014-12-02,13,Mar,2024,HCFP&A,10,2,Dec,2014,...,NaN,NaN,NaN,NaN,2024-03-13,2014-12-02,NaT,CIVIL,0,0
1,Voi,Voi-HCFP&A-127/2016-05-11,14,Mar,2024,HCFP&A,127,11,May,2016,...,NaN,NaN,NaN,NaN,2024-03-14,2016-05-11,NaT,CIVIL,0,1
2,Voi,Voi-HCCA-E014/2023-05-12,14,Feb,2024,HCCA,E014,12,May,2023,...,NaN,NaN,NaN,NaN,2024-02-14,2023-05-12,NaT,CIVIL,0,0
3,Voi,Voi-HCCRC-4/2019-04-09,4,Mar,2024,HCCRC,4,9,Apr,2019,...,NaN,NaN,1.0,NaN,2024-03-04,2019-04-09,NaT,CRIMINAL,0,0
4,Voi,Voi-HCCRC-4/2020-08-31,7,Feb,2024,HCCRC,4,31,Aug,2020,...,NaN,NaN,3.0,NaN,2024-02-07,2020-08-31,NaT,CRIMINAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538204,Kitale,Kitale-HCCRA-E031/2023-04-12,29,Sep,2023,HCCRA,E031,12,Apr,2023,...,NaN,NaN,NaN,NaN,2023-09-29,2023-04-12,NaT,CRIMINAL,0,0
538205,Kitale,Kitale-HCCRA-E016/2023-02-20,29,Sep,2023,HCCRA,E016,20,Feb,2023,...,NaN,NaN,NaN,NaN,2023-09-29,2023-02-20,NaT,CRIMINAL,0,0
538206,Kitale,Kitale-HCCRA-E024/2023-03-23,29,Sep,2023,HCCRA,E024,23,Mar,2023,...,NaN,NaN,NaN,NaN,2023-09-29,2023-03-23,NaT,CRIMINAL,0,0
538207,Kitale,Kitale-HCCA-E051/2023-07-27,29,Sep,2023,HCCA,E051,27,Jul,2023,...,NaN,NaN,NaN,NaN,2023-09-29,2023-07-27,NaT,CIVIL,0,0
